# [AI训练营]PaddleX实现目标检测baseline

手把手教你基于PaddleX实现目标检测。你需要实现以下任务：

> 1. 配置数据集（数据集选择、数据处理）
> 2. 配置模型并训练
> 3. 项目跑通即可达到结业要求

# 一、数据集说明

本项目使用的数据集是：[[AI训练营]目标检测数据集合集](https://aistudio.baidu.com/aistudio/datasetdetail/103743)，包含口罩识别 、交通标志识别、火焰检测、锥桶识别以及中秋元素识别。

该数据集已加载至本环境中，位于：**data/data103743/objDataset.zip**

In [ ]:
# 解压数据集（解压一次即可，请勿重复解压）
!unzip -oq /home/aistudio/data/data103743/objDataset.zip

解压完成后，左侧文件夹处会多一个名为**objDataset**的文件夹，该文件夹下有5个子文件夹：
- **barricade**——Gazebo锥桶检测
- **facemask**——口罩检测
- **fire**——火焰检测
- **MidAutumn**——中秋元素检测
- **roadsign_voc**——交通路标检测

每个子文件夹下有2个文件夹，分别存放着图像（**JPEGImages**）和标注文件（**Annotations**），如下所示：

In [1]:
# 查看数据集文件结构
!tree objDataset -L 2

objDataset
├── barricade
│   ├── Annotations
│   ├── JPEGImages
│   ├── labels.txt
│   ├── test_list.txt
│   ├── train_list.txt
│   └── val_list.txt
├── facemask
│   ├── Annotations
│   └── JPEGImages
├── fire
│   ├── Annotations
│   └── JPEGImages
├── MidAutumn
│   ├── Annotations
│   └── JPEGImages
└── roadsign_voc
    ├── Annotations
    └── JPEGImages

15 directories, 4 files


# 二、数据准备

本基线系统使用的数据格式是PascalVOC格式，开发者基于PaddleX开发目标检测模型时，无需对数据格式进行转换，开箱即用。

但为了进行训练，还需要将数据划分为训练集、验证集和测试集。划分之前首先需要**安装PaddleX**。

In [2]:
# 安装PaddleX
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/


使用如下命令即可将数据划分为70%训练集，20%验证集和10%的测试集。

In [3]:
# 划分数据集
!paddlex --split_dataset --format VOC --dataset_dir objDataset/barricade --val_value 0.2 --test_value 0.1

Dataset Split Done.
Train samples: 364
Eval samples: 104
Test samples: 52
Split files saved in objDataset/barricade


划分完成后，该数据集下会生成**labels.txt**, **train_list.txt**, **val_list.txt**和**test_list.txt**，分别存储类别信息，训练样本列表，验证样本列表，测试样本列表。如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/d29a92b4cfc34b0097ef46dbbc8562af824387889f224948ae49283e0adee19d)

在这里，**你需要将path to dataset部分替换成你选择的数据集路径**。在左侧文件夹处，将鼠标放到你想选择的数据集文件夹上，会出现三个图标，第一个图标表示复制该文件夹路径，点击即可获得该文件夹路径，用这个路径替换path to dataset即可。

![](https://ai-studio-static-online.cdn.bcebos.com/c28ed88586644f64b34709a592fea0b97ec80470c0e041fd9aa6b8da21c8e283)


# 三、数据预处理

在训练模型之前，对目标检测任务的数据进行操作，从而提升模型效果。可用于数据处理的API有：
- **Normalize**：对图像进行归一化
- **ResizeByShort**：根据图像的短边调整图像大小
- **RandomHorizontalFlip**：以一定的概率对图像进行随机水平翻转
- **RandomDistort**：以一定的概率对图像进行随机像素内容变换

更多关于数据处理的API及使用说明可查看文档：
[https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/transforms/det_transforms.html](https://paddlex.readthedocs.io/zh_CN/release-1.3/apis/transforms/det_transforms.html)

In [4]:
from paddlex.det import transforms

# 定义训练和验证时的transforms
# API说明 https://paddlex.readthedocs.io/zh_CN/develop/apis/transforms/det_transforms.html
train_transforms = transforms.Compose([
    # 此处需要补充图像预处理代码
    transforms.Normalize(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomDistort()
])

eval_transforms = transforms.Compose([
    # 此处需要补充图像预处理代码
    transforms.Normalize(),
    transforms.Resize(26)
])

读取PascalVOC格式的检测数据集，并对样本进行相应的处理。

In [5]:
import paddlex as pdx

# 定义训练和验证所用的数据集
# API说明：https://paddlex.readthedocs.io/zh_CN/develop/apis/datasets.html#paddlex-datasets-vocdetection
train_dataset = pdx.datasets.VOCDetection(
    data_dir='objDataset/barricade',
    file_list='objDataset/barricade/train_list.txt',
    label_list='objDataset/barricade/labels.txt',
    transforms=train_transforms,
    shuffle=True)

eval_dataset = pdx.datasets.VOCDetection(
    data_dir='objDataset/barricade',
    file_list='objDataset/barricade/val_list.txt',
    label_list='objDataset/barricade/labels.txt',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


2021-08-15 22:46:35 [INFO]	Starting to read file list from dataset...
2021-08-15 22:46:35 [INFO]	364 samples in file objDataset/barricade/train_list.txt
creating index...
index created!
2021-08-15 22:46:35 [INFO]	Starting to read file list from dataset...
2021-08-15 22:46:36 [INFO]	104 samples in file objDataset/barricade/val_list.txt
creating index...
index created!


需要注意的是：
- **data_dir** (str): 数据集所在的目录路径。
- **file_list** (str): 描述数据集图片文件和对应标注文件的文件路径（文本内每行路径为相对data_dir的相对路径）。
- **label_list** (str): 描述数据集包含的类别信息文件路径。

需要将第二步数据准备时生成的labels.txt, train_list.txt, val_list.txt和test_list.txt配置到以上变量中，如下图所示：

![](https://ai-studio-static-online.cdn.bcebos.com/6462f7811da3436290948e5dde0c497d6ae51bcfe1904e0a863ff032363a4448)


# 四、模型训练

PaddleX目前提供了FasterRCNN和YOLOv3两种检测结构，多种backbone模型。本基线系统以骨干网络为MobileNetV1的YOLOv3算法为例。

In [6]:
# 初始化模型
# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#paddlex-det-yolov3

# 此处需要补充目标检测模型代码
model = pdx.det.YOLOv3(num_classes=len(train_dataset.labels), backbone='MobileNetV1')

In [7]:
# 模型训练
# API说明: https://paddlex.readthedocs.io/zh_CN/develop/apis/models/detection.html#id1
# 各参数介绍与调整说明：https://paddlex.readthedocs.io/zh_CN/develop/appendix/parameters.html

# 此处需要补充模型训练参数
model.train(
    num_epochs=100,
    train_dataset=train_dataset,
    train_batch_size=8,
    eval_dataset=eval_dataset,
    learning_rate=0.000125,
    lr_decay_epochs=[210, 240],
    save_dir='output/yolov3_mobilenetv1')

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/framework.py:706: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif dtype == np.bool:


2021-08-15 22:46:46 [INFO]	Decompressing output/yolov3_mobilenetv1/pretrain/MobileNetV1_pretrained.tar...
2021-08-15 22:46:49 [INFO]	Load pretrain weights from output/yolov3_mobilenetv1/pretrain/MobileNetV1_pretrained.
2021-08-15 22:46:50 [INFO]	There are 135 varaibles in output/yolov3_mobilenetv1/pretrain/MobileNetV1_pretrained are loaded.
2021-08-15 22:46:56 [INFO]	[TRAIN] Epoch=1/100, Step=2/45, loss=16462.138672, lr=0.0, time_each_step=3.18s, eta=4:2:2
2021-08-15 22:46:57 [INFO]	[TRAIN] Epoch=1/100, Step=4/45, loss=5964.205078, lr=0.0, time_each_step=1.8s, eta=2:16:32
2021-08-15 22:46:58 [INFO]	[TRAIN] Epoch=1/100, Step=6/45, loss=13009.943359, lr=1e-06, time_each_step=1.36s, eta=1:43:27
2021-08-15 22:46:59 [INFO]	[TRAIN] Epoch=1/100, Step=8/45, loss=13151.165039, lr=1e-06, time_each_step=1.17s, eta=1:28:54
2021-08-15 22:47:00 [INFO]	[TRAIN] Epoch=1/100, Step=10/45, loss=9019.326172, lr=1e-06, time_each_step=1.03s, eta=1:17:55
2021-08-15 22:47:01 [INFO]	[TRAIN] Epoch=1/100, Step=12

100%|██████████| 13/13 [00:05<00:00,  2.18it/s]


2021-08-15 22:55:03 [INFO]	[EVAL] Finished, Epoch=20, bbox_map=0.0 .
2021-08-15 22:55:03 [INFO]	Model saved in output/yolov3_mobilenetv1/best_model.
2021-08-15 22:55:04 [INFO]	Model saved in output/yolov3_mobilenetv1/epoch_20.
2021-08-15 22:55:04 [INFO]	Current evaluated best model in eval_dataset is epoch_20, bbox_map=0.0
2021-08-15 22:55:10 [INFO]	[TRAIN] Epoch=21/100, Step=2/45, loss=8.009802, lr=0.000113, time_each_step=0.57s, eta=0:34:54
2021-08-15 22:55:11 [INFO]	[TRAIN] Epoch=21/100, Step=4/45, loss=6.470341, lr=0.000113, time_each_step=0.57s, eta=0:34:53
2021-08-15 22:55:12 [INFO]	[TRAIN] Epoch=21/100, Step=6/45, loss=6.778859, lr=0.000113, time_each_step=0.58s, eta=0:34:52
2021-08-15 22:55:13 [INFO]	[TRAIN] Epoch=21/100, Step=8/45, loss=7.666707, lr=0.000113, time_each_step=0.57s, eta=0:34:51
2021-08-15 22:55:14 [INFO]	[TRAIN] Epoch=21/100, Step=10/45, loss=6.859779, lr=0.000114, time_each_step=0.6s, eta=0:34:50
2021-08-15 22:55:15 [INFO]	[TRAIN] Epoch=21/100, Step=12/45, loss

100%|██████████| 13/13 [00:10<00:00,  1.19it/s]


...

2021-08-15 23:11:05 [INFO]	[EVAL] Finished, Epoch=60, bbox_map=0.0 .
2021-08-15 23:11:06 [INFO]	Model saved in output/yolov3_mobilenetv1/epoch_60.
2021-08-15 23:11:06 [INFO]	Current evaluated best model in eval_dataset is epoch_20, bbox_map=0.0
2021-08-15 23:11:11 [INFO]	[TRAIN] Epoch=61/100, Step=2/45, loss=3.902364, lr=0.000125, time_each_step=0.49s, eta=0:18:19
2021-08-15 23:11:12 [INFO]	[TRAIN] Epoch=61/100, Step=4/45, loss=4.3159, lr=0.000125, time_each_step=0.49s, eta=0:18:18
2021-08-15 23:11:13 [INFO]	[TRAIN] Epoch=61/100, Step=6/45, loss=4.283329, lr=0.000125, time_each_step=0.51s, eta=0:18:17
2021-08-15 23:11:14 [INFO]	[TRAIN] Epoch=61/100, Step=8/45, loss=3.925749, lr=0.000125, time_each_step=0.5s, eta=0:18:16
2021-08-15 23:11:15 [INFO]	[TRAIN] Epoch=61/100, Step=10/45, loss=4.637874, lr=0.000125, time_each_step=0.52s, eta=0:18:16
2021-08-15 23:11:16 [INFO]	[TRAIN] Epoch=61/100, Step=12/45, loss=5.193226, lr=0.000125, time_each_step=0.54s, eta=0:18:16
2021-08-15 23:11:17 [INF

100%|██████████| 13/13 [00:05<00:00,  2.50it/s]


2021-08-15 23:18:57 [INFO]	[EVAL] Finished, Epoch=80, bbox_map=0.0 .
2021-08-15 23:18:58 [INFO]	Model saved in output/yolov3_mobilenetv1/epoch_80.
2021-08-15 23:18:58 [INFO]	Current evaluated best model in eval_dataset is epoch_20, bbox_map=0.0
2021-08-15 23:19:02 [INFO]	[TRAIN] Epoch=81/100, Step=2/45, loss=4.060899, lr=0.000125, time_each_step=0.42s, eta=0:7:28
2021-08-15 23:19:03 [INFO]	[TRAIN] Epoch=81/100, Step=4/45, loss=4.535817, lr=0.000125, time_each_step=0.44s, eta=0:7:28
2021-08-15 23:19:04 [INFO]	[TRAIN] Epoch=81/100, Step=6/45, loss=3.337317, lr=0.000125, time_each_step=0.44s, eta=0:7:27
2021-08-15 23:19:05 [INFO]	[TRAIN] Epoch=81/100, Step=8/45, loss=3.196511, lr=0.000125, time_each_step=0.46s, eta=0:7:27
2021-08-15 23:19:06 [INFO]	[TRAIN] Epoch=81/100, Step=10/45, loss=3.837275, lr=0.000125, time_each_step=0.48s, eta=0:7:27
2021-08-15 23:19:07 [INFO]	[TRAIN] Epoch=81/100, Step=12/45, loss=3.498258, lr=0.000125, time_each_step=0.52s, eta=0:7:27
2021-08-15 23:19:08 [INFO]	

100%|██████████| 13/13 [00:05<00:00,  2.49it/s]


2021-08-15 23:26:40 [INFO]	[EVAL] Finished, Epoch=100, bbox_map=0.0 .
2021-08-15 23:26:41 [INFO]	Model saved in output/yolov3_mobilenetv1/epoch_100.
2021-08-15 23:26:41 [INFO]	Current evaluated best model in eval_dataset is epoch_20, bbox_map=0.0


# 五、作业提交

完成以上四个步骤，并且跑通后，恭喜你！你已经学会使用PaddleX训练目标检测模型了！

首先点击右上方的“**文件**”，点击“**导出Notebook为Markdown**”：
![](https://ai-studio-static-online.cdn.bcebos.com/0b502ce690274eee89c285c702a26a72ff012df791eb45b79b7866a97f549e33)

导出后的文件用于上传至GitHub。

下面请点击左侧“版本”，在出现的新界面中点击“生成新版本”，按如下操作即可生成版本，用于提交作业：
![](https://ai-studio-static-online.cdn.bcebos.com/4e58b131f2db45289119dfa90081eea43b781d6894254069b4f02cffa067ffe1)

生成版本后，回到项目主页，点击“**设置为公开**”：
![](https://ai-studio-static-online.cdn.bcebos.com/4a9229e9eb8146169c848327ccb1f85ea30af431eee14aea9c2e6c9aeb364926)
